In [1]:
# %pip install openai
# %pip install openpyxl
# %pip install python-dotenv

from IPython.display import clear_output
clear_output()

In [2]:
import json
import random
import re
import pandas as pd
import numpy as np


# from sklearn.datasets import fetch_20newsgroups

# import nltk
# from nltk.corpus import stopwords
# nltk.download('punkt')
# nltk.download('stopwords')

In [3]:
Sample1 = """
For the PR I believe it's all good, the way to test it is on development mode in looker and make it work basically. The file is clean and all good for deployment.

But in terms of norming and future cases, I'm strongly against keeping this workflow if the dashboard has the possibility of receiving updates. 
I believe the use of LookML Dashboard was used almost correctly in this case, 
but on the case where we need to apply changes in this dashboard or add dimensions/new tiles it's not maintainable to add these things via LookML Dashboards overall, 
based on Looker is also a tool built to actually use the User-Defined features on explores. 
I believe we should follow more on the line where Guilherme explained to us, 
use this process to deploy the dashboard once it's ready and explore the options about changing sources when it's in production (model/views names...). 
On the other hand, Indeed this open us for huge improvements and version control on our dashboards, 
I strongly recommend exploring the possibility to convert this code into a User-Defined dashboard and see how we could keep changing it when its necessary (e.g. a new requirement comes).

(e.g. my opinion: We could talk more about this with the whole team and maybe leave this option open to whoever builds a dashboard if they want to have it as a LookML Dashboard if the dashboard requires it, 
like in this case, but evaluate why do we have to leave it like this; 
in this triage case we only did that and it cause us days of work just to have a dynamic hard coded date in the date selection because the business had hard times adapting with the Looker built-in options, 
we could have explored an option similar to what we do with the timeframes for example, and we never ended up discovering and exploring more LookML dashboards. 
It also worries me that this is a huge cannon to tackle minor requirements.)
Based on this, I think a discussion about it would be the way to go, it's a feature that Looker has, as any other, 
but it's not that straight forward to use, it limits us in other ways but it brings us a lot of other good stuff and robust and powerful opportunities, 
one of it is to keep version control about changes in the dashboard over time, but we have to decide as a team either is it is required/nice to have/cool thing let's use only when we need or desire.
"""

In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_PROJECT = os.getenv("OPENAI_PROJECT_ID")
OPENAI_APIKEY = os.getenv("OPENAI_APIKEY")

# print(OPENAI_PROJECT,OPENAI_APIKEY)

## Authenticate with OpenAI via CURL
# !curl https://api.openai.com/v1/models -H "Authorization: Bearer $OPENAI_APIKEY" -H "OpenAI-Project: $OPENAI_PROJECT"

In [8]:
from openai import OpenAI

client = OpenAI(
    api_key=OPENAI_APIKEY,
    project=OPENAI_PROJECT
)

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful technical assistant, \
                                       specialist in non violent communication and all communication matters, \
                                       with the most updated infomation about IT and data science."}, 
        {"role": "user", "content":   "I'm an a NLP student and I need help with a feedback. \
                                       Tell me about what could be better on the following feedback to be more actionable, \
                                       and also in a scale of 1-10, how do rank this feedback (and explain based on what). Feedback:"+Sample1},
    ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='### Feedback Analysis and Improvement\n\n#### Analysis\n**Clarity and Specificity:**\n- The feedback has a good amount of detailed information, but it is lengthy and somewhat convoluted.\n- There are multiple points and views which may dilute the core message.\n- It is difficult to discern actionable steps from the feedback.\n\n**Actionability:**\n- The feedback proposes a discussion but does not provide a concrete plan for how or when to have this discussion.\n- The use of subjective language like "I believe" and "my opinion" without specific examples or data reduces the persuasiveness.\n- Suggestions are present, but they aren\'t clearly separated from the main text making them hard to extract.\n\n**Positivity and Constructiveness:**\n- The feedback is mostly positive about the existing work but turns cautionary on future implementations.\n- It lacks clear, actionable steps for improvement or alternative workflows.\n\n**Ranking:**\nOn a scale of 1-10, I

In [14]:
#write a file with the response:

with open("resposta1.md", "w") as f:
    f.write(completion.choices[0].message.content)

In [ ]:
# Dataset with studentes 
Sample2 = pd.read_excel('datasets\\finalDataset0.2.xlsx')

Proximos Passos:
- Ver algum método de detecção de hate speech e aplicar neste feedback